### DATA  CHECK FROM AUTHOR'S PROVIDED IDs

In [25]:
!pip3 install twython

In [26]:
from twython import Twython, TwythonError

### DATA EXTRACTION

In [27]:
# Pip install Tweepy if you don't already have the package
!pip install tweepy

# Imports
import tweepy
import pandas as pd
import time

### DDEO

In [28]:
!pip install pyLDAvis

In [29]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from matplotlib import pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [30]:
from gensim.models.phrases import Phrases, Phraser

In [31]:
# read dataset
# we will be using trained dataset to understand how people are reacting
data = pd.read_csv('new_data.csv', encoding='utf-8')

In [32]:
#basic stats
data.describe()

,Tweet_id,Text,Created_at
count,91936,91928,91923
unique,12,71595,59337
top,7.38E+17,RT @TheWeirdWorld: My favorite type of exercis...,2016-06-02 01:58:11+00:00
freq,73303,496,21


In [33]:
### max tweet length
df = pd.DataFrame(data, columns=['Text'])
max_length = df[df['Text'].apply(lambda x: isinstance(x, str))]['Text'].map(lambda x: len(x)).max()
print(max_length)

1778


In [34]:
df = data.dropna(subset=['Text'])

In [35]:
!pip install tweet-preprocessor

In [36]:
import preprocessor as p
df['clean_text'] = df['Text'].apply(lambda text: p.clean(str(text)))
df['clean_text'] = df['clean_text'].apply(lambda text: text.lower())
def remove_contractions(text):
    text = re.sub(r'’', '\'', text)
    
    text = re.sub(r'won\'t', 'will not', text)
    text = re.sub(r'can\'t', 'can not', text)
    
    text = re.sub(r'\'s', ' is', text)
    text = re.sub(r'\'m', ' am', text)
    text = re.sub(r'\'re', ' are', text)
    text = re.sub(r'\'ve', ' have', text)
    text = re.sub(r'\'ll', ' will', text)
    text = re.sub(r'\'d', ' would', text)
    text = re.sub(r'\'t', ' not', text)
    text = re.sub(r'n\'t', ' not', text)
    
    return text

df['clean_text'] = df['clean_text'].apply(remove_contractions)
df['clean_text'] = df['clean_text'].str.replace('[^\w\s]', '')
# Remove whitespaces and empty tweets.
df['clean_text'] = df['clean_text'].apply(lambda text: str(text).strip())
# Remove empty texts.
df = df.drop(df[df['clean_text'] == ''].index)

C:\Users\panka\AppData\Local\Temp\ipykernel_4016\1588321190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = df['Text'].apply(lambda text: p.clean(str(text)))
C:\Users\panka\AppData\Local\Temp\ipykernel_4016\1588321190.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = df['clean_text'].apply(lambda text: text.lower())
C:\Users\panka\AppData\Local\Temp\ipykernel_4016\1588321190.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [37]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\panka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

####Tokenize words and Clean-up text

Gensim’s simple_preprocess() is great for this. Additionally I have set deacc=True to remove the punctuations.

In [39]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_text = df['clean_text']
data_words = list(sent_to_words(data_text))

print(data_words[:1])

[['when', 'the', 'coworkers', 'compliment', 'you', 'on', 'how', 'skinny', 'you', 'lookbut', 'little', 'do', 'they', 'know', 'what', 'your', 'diet', 'really', 'consisted', 'of', 'this', 'weekend']]


In [40]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [41]:
def remove_stopwords(texts):
   return [[word for word in simple_preprocess(str(doc)) 
   if word not in stop_words] for doc in texts]
def make_bigrams(texts):
   return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
   return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
   texts_out = []
   for sent in texts:
      doc = nlp(" ".join(sent))
      texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
   return texts_out

In [42]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [43]:
print(data_lemmatized[:1])

[['coworker', 'compliment', 'skinny', 'little', 'really', 'consist', 'weekend']]


### Create the Dictionary and Corpus needed for Topic Modeling

In [44]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [45]:
id2word[0]

'compliment'

In [46]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('compliment', 1),
  ('consist', 1),
  ('coworker', 1),
  ('little', 1),
  ('really', 1),
  ('skinny', 1),
  ('weekend', 1)]]

In [47]:
#Building LDA Mallet Model
ldamallet = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=20, id2word=id2word, chunksize=100)

In [48]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(13,
  [('m', 0.44907677),
   ('ugly', 2.1493071e-05),
   ('mad', 2.1488764e-05),
   ('fucking', 2.1487202e-05),
   ('sad', 2.1487052e-05),
   ('lay', 2.1485226e-05),
   ('joint', 2.1485033e-05),
   ('headddd', 2.148476e-05),
   ('smash', 2.148476e-05),
   ('ting', 2.148476e-05)]),
 (6,
  [('take', 0.23625834),
   ('let', 0.099893816),
   ('gain', 0.075522535),
   ('hot', 0.06574743),
   ('hard', 0.058067698),
   ('energy', 0.03745173),
   ('stay', 0.033659354),
   ('health', 0.030344998),
   ('offer', 0.026492633),
   ('word', 0.025715021)]),
 (14,
  [('lose', 0.29577115),
   ('loss', 0.24989974),
   ('weight', 0.15823603),
   ('diet', 0.15593594),
   ('http', 0.042805124),
   ('add', 0.02078356),
   ('body', 3.0043843e-06),
   ('plan', 2.9911196e-06),
   ('fast', 2.9907749e-06),
   ('made_possible', 2.9897737e-06)]),
 (15,
  [('know', 0.39562467),
   ('today', 0.14782068),
   ('thing', 0.10657205),
   ('coffee', 0.032813713),
   ('complete', 0.026881551),
   ('hear', 0.026729466),
 

In [49]:
#How to find the optimal number of topics for LDA?
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [50]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=300, step=4)

In [1]:
# Show graph
limit=300; start=2; step=4;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.4177
Num Topics = 6  has Coherence Value of 0.3706
Num Topics = 10  has Coherence Value of 0.3989
Num Topics = 14  has Coherence Value of 0.4095
Num Topics = 18  has Coherence Value of 0.4055
Num Topics = 22  has Coherence Value of 0.4091
Num Topics = 26  has Coherence Value of 0.4349
Num Topics = 30  has Coherence Value of 0.469
Num Topics = 34  has Coherence Value of 0.4894
Num Topics = 38  has Coherence Value of 0.5162
Num Topics = 42  has Coherence Value of 0.5193
Num Topics = 46  has Coherence Value of 0.5298
Num Topics = 50  has Coherence Value of 0.5265
Num Topics = 54  has Coherence Value of 0.5141
Num Topics = 58  has Coherence Value of 0.5061
Num Topics = 62  has Coherence Value of 0.4984
Num Topics = 66  has Coherence Value of 0.4765
Num Topics = 70  has Coherence Value of 0.45
Num Topics = 74  has Coherence Value of 0.4154
Num Topics = 78  has Coherence Value of 0.399
Num Topics = 82  has Coherence Value of 0.3683
Num Topics = 86  ha

In [ ]:
# Select the model and print the sub-topics
optimal_model = model_list[57]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(63,
  '0.000*"comman" + 0.000*"selfdelusion" + 0.000*"actuarial" + '
  '0.000*"lowimpact" + 0.000*"corrupt" + 0.000*"amptransmitte" + '
  '0.000*"stampede" + 0.000*"duckling" + 0.000*"wee" + 0.000*"plsif"'),
 (191,
  '0.000*"comman" + 0.000*"selfdelusion" + 0.000*"actuarial" + '
  '0.000*"lowimpact" + 0.000*"corrupt" + 0.000*"amptransmitte" + '
  '0.000*"stampede" + 0.000*"duckling" + 0.000*"wee" + 0.000*"plsif"'),
 (76,
  '0.000*"comman" + 0.000*"selfdelusion" + 0.000*"actuarial" + '
  '0.000*"lowimpact" + 0.000*"corrupt" + 0.000*"amptransmitte" + '
  '0.000*"stampede" + 0.000*"duckling" + 0.000*"wee" + 0.000*"plsif"'),
 (131,
  '0.000*"comman" + 0.000*"selfdelusion" + 0.000*"actuarial" + '
  '0.000*"lowimpact" + 0.000*"corrupt" + 0.000*"amptransmitte" + '
  '0.000*"stampede" + 0.000*"duckling" + 0.000*"wee" + 0.000*"plsif"'),
 (106,
  '0.000*"comman" + 0.000*"selfdelusion" + 0.000*"actuarial" + '
  '0.000*"lowimpact" + 0.000*"corrupt" + 0.000*"amptransmitte" + '
  '0.000*"stampede"